#### Instructions:  
1. Libraries allowed: **Python basic libraries, numpy, pandas, scikit-learn (only for data processing), pytorch, and ClearML.**
2. Show all outputs.
3. Submit jupyter notebook and a pdf export of the notebook. Check canvas for detail instructions for the report.
4. Below are the questions/steps that you need to answer. Add as many cells as needed.

## Task 2: Finetuning a pretrained NN
Do transfer learning with ResNet18 and compare peforamnce with the hyperparamter-tuned network.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Dataset paths
train_dir = "/content/drive/MyDrive/Car_Brand_Logos/Train"
test_dir = "/content/drive/MyDrive/Car_Brand_Logos/Test"

# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Load datasets
train_dataset = ImageFolder(train_dir, transform=transform)
test_dataset = ImageFolder(test_dir, transform=transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
# Load pretrained ResNet18
model = models.resnet18(pretrained=True)

# Modify the final layer for the number of classes in your dataset
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 120MB/s]


In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)


In [ ]:
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

    accuracy = 100.0 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Accuracy: {accuracy:.2f}%")


Epoch [1/5], Loss: 39.2178, Accuracy: 59.69%
Epoch [2/5], Loss: 14.5885, Accuracy: 90.16%
Epoch [3/5], Loss: 8.6790, Accuracy: 94.14%
Epoch [4/5], Loss: 6.0747, Accuracy: 95.94%
Epoch [5/5], Loss: 4.5421, Accuracy: 97.34%


In [ ]:
# Evaluate the model on test data
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average="weighted")
recall = recall_score(all_labels, all_preds, average="weighted")
f1 = f1_score(all_labels, all_preds, average="weighted")

print("Evaluation Metrics for ResNet18:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Evaluation Metrics for ResNet18:
Accuracy: 0.8500
Precision: 0.8497
Recall: 0.8500
F1 Score: 0.8491


### Discussion
Provide a comparative analysis.

* The ResNet18 model benefits from pretraining on ImageNet, leveraging feature extraction capabilities and robust parameter initialization. The simpler custom CNN models struggled to achieve similar performance due to their limited architecture.
* The ResNet18 model benefits from pretraining on ImageNet, leveraging feature extraction capabilities and robust parameter initialization. The simpler custom CNN models struggled to achieve similar performance due to their limited architecture.
* The loss reduction for ResNet18 was far more pronounced compared to the custom models. This indicates better optimization and fit to the data within the same number of epochs.
* For real-world applications, where datasets are often smaller and training time is constrained, transfer learning with pre-trained networks like ResNet18 proves to be more efficient and effective.


ResNet18 Fine-Tuned Model (Transfer Learning):
* Parameters:
1. batch_size=32,
2. epochs=5
* Pretrained ResNet18 with ImageNet weights.
* Final layer modified for the dataset classes.
* Performance:
1. Final training loss: 4.5421
2. Final training accuracy: 97.34%
* Test Evaluation Metrics:
1. Accuracy: 85.00%
2. Precision: 84.97%
3. Recall: 85.00%
4. F1 Score: 84.91%

Custom Model (without learning rate decay):
* Parameters:
1. batch_size=4,
2. epochs=5

1. num_layers: 1.0
2. num_filters: 4.0
3. learning_rate: 0.01
4. momentum: 0.524
5. weight_decay: 0.0005

* Performance:
1. Final training loss: 31.3095
2. Final training accuracy: 94.62%
* Test Evaluation Metrics:
1. Accuracy: 0.3500
2. Precision: 0.3468
3. Recall: 0.3500
4. F1 Score: 0.3308

* For tasks requiring higher accuracy and better generalization, fine-tuning pretrained models such as ResNet18 is highly recommended.
* Custom models might be suitable for lightweight, resource-constrained environments.